In [7]:
import pandas as pd
import folium
from folium import plugins
from folium.features import CustomIcon


df1 = pd.read_csv('./data/folium/달서구가로등.csv', encoding='cp949')
df2 = pd.read_csv('./data/folium/달서구CCTV.csv', encoding='cp949')

df.drop(['위도'], axis=1,inplace=True)
df.drop(['경도'], axis=1,inplace=True)

df1['가로등위도']=df1['위도']
df1['가로등경도']=df1['경도']
df1['구분']='가로등'

df2['CCTV위도']=df2['위도']
df2['CCTV경도']=df2['경도']
df2['구분']='CCTV'

df_e = df[df['구분']=='가로등'][:]
df_f = df[df['구분']=='CCTV'][:]


df = pd.concat([df1,df2])
df.fillna(0, inplace=True)

m = folium.Map(
    location =[35.82847171385931, 128.52935555965178],
    zoom_start=13,
    tiles = 'CartoDB positron'
)


gs = folium.FeatureGroup(name = '모두보기')
m.add_child(gs)

g1 = plugins.FeatureGroupSubGroup(group=gs, name='가로등')
m.add_child(g1)

g2 = plugins.FeatureGroupSubGroup(group=gs, name='CCTV')
m.add_child(g2)

cluster1 = plugins.MarkerCluster()

for i in range(df_e.shape[0]):
    
    가로등번호 = df_e.iloc[i]['가로등번호']
    cluster1.add_child(
        folium.CircleMarker(
            location= [df_e.iloc[i]['가로등위도'], df_e.iloc[i]['가로등경도']],
            popup=f'<div style = "width : 120px"><strong>가로등</strong><br>'
                  +f'가로등 번호 : {가로등번호}',
            tooltip = '가로등 보기',
            color='darkyellow',
            radius = 35,
            fill = True,
            fill_color = 'yellow'

        )
    ).add_to(g1)

cluster2 = plugins.MarkerCluster()

for i in range(df_f.shape[0]):
    icon_img = './data/folium/cctv.png'
    icon = CustomIcon(
        icon_img,
        icon_size=(40,40),
        popup_anchor=(30,-30)
    )
        
    설치연도 = df_f.iloc[i]['설치연도']
    cluster2.add_child(
        folium.Marker(
            location= [df_f.iloc[i]['CCTV위도'], df_f.iloc[i]['CCTV경도']],
            popup=f'<div style = "width : 120px"><center><strong>CCTV</strong></center><br>'
                 +f'설치연도 : {설치연도}</div>',
            tooltip = 'CCTV 보기',
            icon=icon
        )
    ).add_to(g2)
    
    
folium.LayerControl(collapsed=False).add_to(m)
minimap = plugins.MiniMap()
m.add_child(minimap)
m.save('./data/folium/김도아.html')